1. Project Setup

In [1]:
!pip install websockets pandas numpy scikit-learn ipywidgets

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

 2. Connecting to OKX via WebSocket

In [4]:
import asyncio, websockets, json

WS_URL = "wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP"

async def stream_orderbook():
    async with websockets.connect(WS_URL) as ws:
        async for message in ws:
            data = json.loads(message)
            # for now, just print the timestamp
            print("Tick at", data["timestamp"])
            break  # remove this later!


In [5]:
await stream_orderbook()

Tick at 2025-05-14T11:16:31Z


 3. Parsing & Storing Order-Book Data

In [6]:
import numpy as np

def parse_orderbook(data):
    asks = np.array(data["asks"], dtype=float)
    bids = np.array(data["bids"], dtype=float)
    return bids, asks

# --- Dummy tick to test your parser ---
sample_data = {
    "timestamp": "2025-05-14T12:00:00Z",
    "asks": [["95448.0", "2.05"], ["95449.0", "5.00"]],
    "bids": [["95445.4", "1104.23"], ["95445.3", "0.02"]],
}

bids, asks = parse_orderbook(sample_data)
print("Best bid:", bids[0], "Best ask:", asks[0])


Best bid: [95445.4   1104.23] Best ask: [9.5448e+04 2.0500e+00]


In [7]:
import asyncio, websockets, json, numpy as np

WS_URL = "wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP"

def parse_orderbook(data):
    asks = np.array(data["asks"], dtype=float)
    bids = np.array(data["bids"], dtype=float)
    return bids, asks

async def stream_and_parse():
    async with websockets.connect(WS_URL) as ws:
        async for message in ws:
            data = json.loads(message)
            bids, asks = parse_orderbook(data)        # ← data is defined here
            print("Best bid:", bids[0], "Best ask:", asks[0])
            break  # remove this once you’re confident it works

# In Jupyter, just await your coroutine:
await stream_and_parse()


Best bid: [103870.      239.23] Best ask: [103870.1    1285.66]


 4. Computing Simple Outputs First

In [8]:
def compute_mid_and_spread(bids, asks):
    best_bid, _ = bids[0]
    best_ask, _ = asks[0]
    mid = (best_bid + best_ask) / 2
    spread = best_ask - best_bid
    return mid, spread


5. Building First “UI” with ipywidgets

In [9]:



# Input widgets
asset = widgets.Dropdown(options=["BTC-USDT", "ETH-USDT"], description="Asset:")
quantity = widgets.FloatText(value=100, description="Qty (USD):")
volatility = widgets.FloatSlider(value=0.5, min=0, max=2, step=0.01, description="Volatility:")
fee_tier = widgets.Dropdown(options=["0.02%", "0.05%"], description="Fee tier:")

# Output areas
out_mid = widgets.Label(value="Mid-price: ")
out_spread = widgets.Label(value="Spread: ")

display(asset, quantity, volatility, fee_tier, out_mid, out_spread)



Dropdown(description='Asset:', options=('BTC-USDT', 'ETH-USDT'), value='BTC-USDT')

FloatText(value=100.0, description='Qty (USD):')

FloatSlider(value=0.5, description='Volatility:', max=2.0, step=0.01)

Dropdown(description='Fee tier:', options=('0.02%', '0.05%'), value='0.02%')

Label(value='Mid-price: ')

Label(value='Spread: ')

In [10]:
# 1) Make sure your compute function is defined and run
def compute_mid_and_spread(bids, asks):
    best_bid, _ = bids[0]
    best_ask, _ = asks[0]
    mid = (best_bid + best_ask) / 2
    spread = best_ask - best_bid
    return mid, spread

# 2) Suppose you already have bids, asks arrays:
bids = np.array([[95445.4, 1104.23], [95445.3, 0.02]])
asks = np.array([[95448.0,   2.05], [95449.0, 5.00]])

# 3) Compute mid & spread
mid, spread = compute_mid_and_spread(bids, asks)

# 4) Now update your widgets
out_mid.value    = f"Mid-price: {mid:.2f}"
out_spread.value = f"Spread:   {spread:.2f}"


6. Adding Slippage, Fees & Impact Models

Slippage model (simple linear regression stub):

In [11]:
# Cell: Slippage model setup
import numpy as np
from sklearn.linear_model import LinearRegression

# 1) Prepare your dummy training data
#    Each row is [volatility], each target is slippage percentage
X_train = np.array([[0.01], [0.02], [0.03]])       # volatilities
y_train = np.array([0.1,  0.2,   0.35 ])           # slippage %’s

# 2) Fit the linear regression
slippage_model = LinearRegression().fit(X_train, y_train)

# 3) Define your helper function
def expected_slippage(vol, qty_usd):
    # Predict slippage percentage from volatility
    pct = slippage_model.predict(np.array([[vol]]))[0]
    return pct * qty_usd / 100


In [12]:
# Pull the float values out of the widgets
vol_val = volatility.value
qty_val = quantity.value

# Now both are real numbers, so your model will accept them
slip_cost = expected_slippage(vol_val, qty_val)
print(f"Expected slippage cost: ${slip_cost:.2f}")


Expected slippage cost: $6.22


In [13]:
import ipywidgets as widgets
from IPython.display import display

# Create the slippage output label
out_slippage = widgets.Label(value="Slippage: ")
display(out_slippage)

slip_cost = expected_slippage(volatility.value, quantity.value)

out_slippage.value = f"Slippage: ${slip_cost:.2f}"


Label(value='Slippage: ')

Fees (rule-based):

In [14]:
def expected_fees(qty_usd, tier_str):
    tier_pct = float(tier_str.strip("%")) / 100
    return qty_usd * tier_pct


Almgren–Chriss impact (very simplified):

In [15]:
def market_impact(qty_usd, vol):
    # placeholder: impact = k * qty * vol
    k = 1e-4
    return k * qty_usd * vol


Net cost = slippage + fees + impact.

7. Maker/Taker Prediction

In [16]:
from sklearn.linear_model import LogisticRegression
X = np.random.rand(100,2)  # e.g. [volatility, qty]
y = (X[:,0]*10 + X[:,1] > 5).astype(int)  # fake label
mt_model = LogisticRegression().fit(X, y)

def maker_taker_pct(vol, qty):
    prob = mt_model.predict_proba([[vol, qty]])[0][1]
    return prob  # percent of maker trades


In [17]:
# 1) Ensure you have the widget
out_mt = widgets.Label(value="Maker %: ")
display(out_mt)

# 2) Pull real numbers from your controls
vol_val = volatility.value
qty_val = quantity.value

# 3) Run prediction and update
mt_pct = maker_taker_pct(vol_val, qty_val) * 100
out_mt.value = f"Maker %: {mt_pct:.1f}"


Label(value='Maker %: ')

In [18]:
import ipywidgets as widgets
from IPython.display import display

out_latency = widgets.Label(value="Latency: ")
display(out_latency)


Label(value='Latency: ')

8. Measuring Latency

In [19]:
import time

start = time.time()
# parse + compute + update UI
latency = time.time() - start
out_latency.value = f"Latency: {latency*1000:.1f} ms"


In [20]:




import ipywidgets as widgets
from IPython.display import display, clear_output

# Input controls
asset       = widgets.Dropdown(options=["BTC-USDT","ETH-USDT"], description="Asset:")
quantity    = widgets.FloatText(value=100, description="Qty (USD):")
volatility  = widgets.FloatSlider(value=0.5, min=0, max=2, step=0.01, description="Vol:")
fee_tier    = widgets.Dropdown(options=["0.02%","0.05%"], description="Fee tier:")

# Output labels
out_mid        = widgets.Label(value="Mid-price: ")
out_spread     = widgets.Label(value="Spread: ")
out_slippage   = widgets.Label(value="Slippage: ")
out_fees       = widgets.Label(value="Fees: ")
out_impact     = widgets.Label(value="Impact: ")
out_netcost    = widgets.Label(value="Net cost: ")
out_mt         = widgets.Label(value="Maker %: ")
out_latency    = widgets.Label(value="Latency: ")

# Display all
display(widgets.VBox([
    widgets.HBox([asset, quantity, volatility, fee_tier]),
    widgets.VBox([out_mid, out_spread, out_slippage, out_fees,
                  out_impact, out_netcost, out_mt, out_latency])
]))


In [21]:
import numpy as np
import json
from sklearn.linear_model import LinearRegression, LogisticRegression

# 2.1) Parse raw JSON → bids/asks arrays
def parse_orderbook(data):
    asks = np.array(data["asks"], dtype=float)
    bids = np.array(data["bids"], dtype=float)
    return bids, asks

# 2.2) Mid & spread
def compute_mid_and_spread(bids, asks):
    best_bid, _ = bids[0]
    best_ask, _ = asks[0]
    mid    = (best_bid + best_ask) / 2
    spread = best_ask - best_bid
    return mid, spread

# 2.3) Slippage model (pre-trained on dummy data)
X = np.array([[0.01],[0.02],[0.03]])
y = np.array([0.1,0.2,0.35])
slippage_model = LinearRegression().fit(X,y)
def expected_slippage(vol, qty):
    pct = slippage_model.predict([[vol]])[0]
    return pct * qty / 100

# 2.4) Fees
def expected_fees(qty, tier_str):
    pct = float(tier_str.strip("%"))/100
    return qty * pct

# 2.5) Market impact (simple stub)
def market_impact(qty, vol):
    return 1e-4 * qty * vol

# 2.6) Maker/Taker model (dummy‐trained)
mt_model = LogisticRegression().fit(
    np.random.rand(100,2), 
    (np.random.rand(100)>0.5).astype(int)
)
def maker_taker_pct(vol, qty):
    return mt_model.predict_proba([[vol,qty]])[0][1]


In [22]:
import asyncio, websockets, time

WS_URL = "wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP"

async def run_simulator():
    async with websockets.connect(WS_URL) as ws:
        async for message in ws:
            tick_start = time.time()
            
            # 3.1) Parse JSON
            data = json.loads(message)
            bids, asks = parse_orderbook(data)
            
            # 3.2) Compute metrics
            mid, spread = compute_mid_and_spread(bids, asks)
            slip   = expected_slippage(volatility.value, quantity.value)
            fees   = expected_fees(quantity.value, fee_tier.value)
            impact = market_impact(quantity.value, volatility.value)
            net    = slip + fees + impact
            mt_pct = maker_taker_pct(volatility.value, quantity.value) * 100
            
            # 3.3) Update UI
            out_mid.value      = f"Mid-price: {mid:.2f}"
            out_spread.value   = f"Spread: {spread:.2f}"
            out_slippage.value = f"Slippage: ${slip:.2f}"
            out_fees.value     = f"Fees: ${fees:.2f}"
            out_impact.value   = f"Impact: ${impact:.2f}"
            out_netcost.value  = f"Net cost: ${net:.2f}"
            out_mt.value       = f"Maker %: {mt_pct:.1f}"
            
            # 3.4) Measure & display latency
            latency = (time.time() - tick_start)*1000
            out_latency.value = f"Latency: {latency:.1f} ms"
            
            # 3.5) For testing, stop after 30 updates
            run_simulator.counter += 1
            if run_simulator.counter >= 30:
                break

# attach a counter attribute
run_simulator.counter = 0


In [23]:
await run_simulator()


In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 1) Create input widgets
quantity    = widgets.FloatText(description="Quantity (USD):", value=100)
volatility  = widgets.FloatSlider(description="Volatility:", min=0, max=1, step=0.01, value=0.1)
fee_tier    = widgets.Dropdown(options=[("VIP 0", 0.002), ("VIP 1", 0.0015)], description="Fee Tier:", value=0.002)

# 2) Create output widgets (just labels to hold values)
mid_price_o = widgets.Label(value="Mid-Price: —")
spread_o    = widgets.Label(value="Spread: —")
slip_o      = widgets.Label(value="Slippage: —")
fees_o      = widgets.Label(value="Fees: —")
impact_o    = widgets.Label(value="Impact: —")
mt_o        = widgets.Label(value="Maker/Taker %: —")
lat_o       = widgets.Label(value="Latency: —")

# 3) Define the update function
def update_metrics(change=None):
    # compute your metrics here; replace the dummy math with your real functions
    mid   = 95000 + quantity.value * 0.1
    spr   = 0.5 + volatility.value * 0.2
    slip  = 0.01 * quantity.value
    fees  = quantity.value * fee_tier.value
    imp   = slip * 0.5
    mtpct = 100 * (1 - fee_tier.value / 0.002)  # dummy
    lat   = 0.001                                # dummy latency
    
    # update each output widget’s .value
    mid_price_o.value = f"Mid-Price: {mid:.2f}"
    spread_o   .value = f"Spread: {spr:.4f}"
    slip_o     .value = f"Slippage: {slip:.4f}"
    fees_o     .value = f"Fees: {fees:.4f}"
    impact_o   .value = f"Impact: {imp:.4f}"
    mt_o       .value = f"Maker/Taker %: {mtpct:.1f}%"
    lat_o      .value = f"Latency: {lat*1000:.2f} ms"

# 4) Hook the update to each input
for w in (quantity, volatility, fee_tier):
    w.observe(update_metrics, names='value')

# 5) Display everything in a layout
inputs  = widgets.VBox([quantity, volatility, fee_tier])
outputs = widgets.VBox([mid_price_o, spread_o, slip_o, fees_o, impact_o, mt_o, lat_o])
display(widgets.HBox([inputs, outputs]))

# 6) Fire once to populate initial values
update_metrics()
